In [ ]:
! [ -e /content ] && pip install -Uqq fastbook


In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
import os
import json
from fastbook import search_images_bing, download_url

if uploaded:
  api_key = json.loads(uploaded.get('colab_keys.json')).get('AZURE_API_KEY')
else:
  api_key = os.environ.get("AZURE_API_KEY", 'XXX')


In [ ]:
from fastai.vision.all import *
import fastbook as fb